In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-4.9 g++-4.9 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

In [0]:
!/usr/local/cuda/bin/nvcc --version

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [0]:
%load_ext nvcc_plugin

In [6]:
%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

Hello world



In [0]:
%%cu
#include "reference_calc.cpp"
#include "utils.h"
#include <stdio.h>

__global__
void rgba_to_greyscale(const uchar4* const rgbaImg,unsigned char* const greyImg,int numRows, int numCols)
{
  
      int indxx = threadIdx.x;  
      int indxy = threadIdx.y;
      int b_indxx = blockIdx.x;
      int b_indxy = blockIdx.y;
      
      int b_dimx = blockDim.x;
      int b_dimy = blockDim.y; 
      int g_dimx = gridDim.x;
      int g_dimy = gridDim.y;
      
      int realx_pos = b_dimx * b_indxx + indxx;
      int realy_pos = b_dimy * b_indxy + indxy;
          
      int off =  realy_pos * (b_dimx * g_dimx) + realx_pos;
      
      uchar4 rgba = rgbaImg[off]; 
      float channelSum = .299f * rgba.x + .587f * rgba.y + .114f * rgba.z;
      greyImg[off] = channelSum; 
    
}

void convert_to_greyscale(const uchar4 * const h_rgbaImg, uchar4 * const d_rgbaImg, unsigned char* const d_greyImg, size_t numRows, size_t numCols)
{
  
  const dim3 blockSize(numRows/16+1, numCols/16+1, 1); 
  const dim3 gridSize( 16, 16, 1);
  rgba_to_greyscale<<<gridSize, blockSize>>>(d_rgbaImg, d_greyImg, numRows, numCols);
  
  cudaDeviceSynchronize(); 
  checkCudaErrors(cudaGetLastError());
}
